Real Time Radar Parameter Optimizer
===================================

Carson Anderson & Calvin Henggeler  
ASEN 5264 Decision Making Under Uncertainty - Spring 2025  
Semester Project  

In [118]:
using POMDPs
using POMDPTools: DiscreteUpdater, ImplicitDistribution, RolloutSimulator
using QuickPOMDPs: QuickPOMDP
using POMDPTesting: has_consistent_distributions
using QMDP: QMDPSolver
using Plots
using Statistics: std
using POMDPPolicies: alphavectors, FunctionPolicy
using Random, Distributions
using ParticleFilters
include("radarFunctions.jl")
include("radarSimulator.jl")

plot_wavefield (generic function with 1 method)

## 1. Create Smart Radar POMDP

### Previoius POMDP

In [119]:
# radar_tracking_calvin = QuickPOMDP(
#     
#     # --- Environment Parameters ---
#     c = 3*10^8  # Speed of light
#     f = 1e9     # Frequency
#     λ = c/f     # Wavelength
#     x_coverage_area = 100e3     # 100 km x 100 km area
#     y_coverage_area = 100e3
#     radar_location = (50e3, 0)  # Radar location (x, y) in meters
# 
#     # --- STATE SPACE ---
#     states = nothing,     # We'll model state generation implicitly
#     
#     # --- ACTION SPACE ---
#     actions = [
#         # Example discretized radar settings (fill in realistic values)
#         # start with a range of differeent realistic settings
#         (beamwidth=5, pulse_length=0.000001, Tipp= 0.001) # , rotation_rate=5),
#         (beamwidth=5, pulse_length=0.000001, Tipp= 0.001) # , rotation_rate=10),
#         (beamwidth=5, pulse_length=0.000002, Tipp= 0.001) # , rotation_rate=5),
#         (beamwidth=5, pulse_length=0.000002, Tipp= 0.001) # , rotation_rate=10),
#     ],
#     # 
#     
#     obstype = Vector{Float64},   # Observation: [power, phase, delay, doppler]
# 
#     gen = function (s, a, rng)
#         # s = (x, y, vx, vy),   extract sate information
#         x, y, vx, vy = s
#         true_range = sqrt((xp - radar_location[1])^2 + (yp - radar_location[2])^2)
# 
#         #TODO: set noise parameters based on radar settings, then add noise to obsercations measurments
# 
# 
#         # Observation Measurments
#         received_power  = radar_return_power(λ, true_range)
#         view_angle      = radar_view_angle(x, y, radar_location)
#         delay           = radar_return_delay(true_range)
#         doppler_obs     = radar_doppler(s)
#         measurments = [received_power, view_angle, delay, doppler_obs]
# 
#         # solve for the observed state
#         o = radar_meas_2_obs_sate()
# 
#         # REWARD: - (tracking error) - (velocity_error) - (action cost)
#         xo, yo, vxo, vyo = o
#         tracking_error = sqrt((x - xo)^2 + (y - yo)^2)              # Euclidean distance
#         velocity_error = sqrt(vx^2 + vy^2) - sqrt(vxo^2 + vyo^2)    # difference of absolute velocities
#         r = -tracking_error - velocity_error
#         # peanalties for actions should realistically be seen in greater measurement noise
# 
#         # TRANSITION MODEL: Target motion (simple linear)
#         dt = T_ipp * 10     # time step
#         xp = x + vx*dt      # + noise_std*randn(rng)
#         yp = y + vy*dt      # + noise_std*randn(rng)
#                             # keep velocities
#         sp = (xp, yp, vx, vy)
# 
#         return (sp=sp, o=o, r=r)
#     end,
# 
#     # TINITIAL STATE GENERATION
#     # Target starts at either far side edge of the coverage area (x ∈ [0, 100e3], y ∈ [0, 100e3])
#     # target will aways start at y = 50e3, ending at a random y at the other side, gives relative heading
#     # Absolute velocity is uniformy distributed from 50 - 300 m/s, v_x and v_x derived from heading and absolute velocity
#     
#     function generate_initial_target(rng::AbstractRNG=Random.default_rng())
#         # 1. Start on left or right edge
#         start_x = rand(rng, [0.0, 100e3])  # 0 or 100,000 m
#     
#         # 2. Always start at y = 50 km
#         start_y = 50e3  # meters
#     
#         # 3. Pick random end y on the opposite side
#         end_y = rand(rng) * 100e3  # [0, 100_000] m
#     
#         # 4. Compute heading angle from start to end
#         dx = (start_x == 0.0) ? 100e3 : -100e3
#         dy = end_y - start_y
#         heading_rad = atan(dy, dx)  # angle in radians
#     
#         # 5. Choose absolute speed [50, 300] m/s
#         speed = rand(rng, Uniform(50.0, 300.0))
#     
#         # 6. Derive velocity components
#         vx = speed * cos(heading_rad)
#         vy = speed * sin(heading_rad)
#     
#         return (x=start_x, y=start_y, vx=vx, vy=vy)b
#     end
#     
#     initiatial_state generate_initial_target()
# 
#     discount = 0.99,
# 
#     isterminal = s -> false  # Tracking problem — no natural terminal state 
#     # TODO: termainal states will be when the target reaches the end of the radar simulator
# )


### Environment Parameters

In [120]:
    # ===================
    # --- ENVIRONMENT ---
    # ===================

    # Environment Grid
    x_max_size = 10000.0
    y_max_size = 10000.0    
    divisions  = 500
    x = collect(LinRange(-x_max_size, x_max_size, divisions))
    y = collect(LinRange(-y_max_size, y_max_size, divisions))
    global env = RadarEnvironment(x, y)

    # Receiver (same as transmitter)
    pos_rx = SVector(0.0, 0.0)
    snapped_pos_rx = snap_to_grid(env.grid_x, pos_rx)
    rx = Receiver(snapped_pos_rx, Float64[], 3.0)
    add_receiver!(env, rx)

1-element Vector{Receiver}:
 Receiver([20.04008016031912, 20.04008016031912], Float64[], 3.0)

### Radar POMDP

In [121]:
radar = QuickPOMDP(
    
    # ====================
    # --- ACTION SPACE ---
    # ====================

    statetype = Vector{Float64},  # State: [x, y]
    obstype = Vector{Float64},    # Observation: [belief_x, belief_y]

    discount = 0.95,

    actions = [(steering_angle, beamwidth, power) for steering_angle in 0:5:355 for beamwidth in 5:5:45 for power in [1, 10, 50, 100]],
    
    # ========================
    # --- GENERATIVE MODEL ---
    # ========================
    # obstype = Vector{Float64},   # Observation: [steering_angle, range, peak_power]

    # gen = function(s, a, rng)
    #     steer_ang, beamwidth, tx_power = a  # Extract action parameters
    #     xpos, ypos = s                      # Extract state parameters

    #     # Create trnasmitter object with action parameters
    #     # PointTransmitter(position, frequency, power, tx_time, pulse_width, steering_angle, beamwidth, is_isotropic)
    #     tx = PointTransmitter(SVector(0.0, 0.0), 1e9, tx_power, 0.0, 0.6, steer_ang, beamwidth, false)
    #     add_transmitter!(env, tx)

    #     # Act on the Environment
    #     # ----------------------
    #     for t = 1:95 
    #         # perform search
    #         step!(env, 1.0)
    #     end
    #     rm_transmitter!(env, tx)  # Remove the transmitter after use

    #     # Retreive observation/ measurments
    #     return_powers_arr = env.receivers[1].received_power + broadcast(abs, (1e-11)*randn(length(rx.received_power))) # make into normal dist
    #     return_power, return_delay = findmax(return_powers_arr[])
    #     range = radar_range(return_delay * 10^-3, noise_std = 0.6*10^-6 ) # return time in ms, noise_std in μs

    #     # Calculate a belif state from noisy observation
    #     beleif_state = boresight_polar_2_cartesian(range, steer_ang + randn()*beamwidth/2)

    #     # State Transition
    #     # ------------------
    #     sp = s
        
    #     # Reward function
    #     # --------------- 
    
    #     # TODO: vectorize this
    #     # TODO: use normilization function
    #     tracking_error = sqrt((xpos - beleif_state[1])^2 + (ypos - beleif_state[2])^2)  # Euclidean distance

    #     if tracking_error < 100

    #         reward = 100
        
    #     else
    #         power_cost = tx_power  # Cost of power used (arbitrary units)

    #         # normaize costs to be evenly weighted
    #         norm_tracking_error = tracking_error / (sqrt(2 * (x_max_size^2 + y_max_size^2)))  # Normalize to max distance in grid
    #         power_cost = power_cost / (100e3)  # Normalize to max power in grid

    #         reward = - (100 * power_cost) - (100 * norm_tracking_error)
    #     end

    #     return (sp=sp, o=[steering_angle, peak_power, range], r=reward)

    # end,

    transition = function (s, a)
        # For now, this is identity — modify for real motion later
        return s
    end,

    observation = function (a, s)
        steer_ang, beamwidth, tx_power = a
        xpos, ypos = s
    
        # Build and inject transmitter
        tx = PointTransmitter(SVector(0.0, 0.0), 1e9, tx_power, 0.0, 0.6, steer_ang, beamwidth, false)
        add_transmitter!(env, tx)
    
        # Propagate environment
        for t = 1:95
            step!(env, 1.0)
        end
        rm_transmitter!(env, tx)
    
        # Simulated radar measurements
        powers = env.receivers[1].received_power
        noisy_powers = powers .+ abs.(1e-11 .* randn(length(powers)))
        return_power, return_delay = findmax(noisy_powers)
    
        range = radar_range(return_delay * 1e-3, noise_std = 0.6e-6)  # ms → s
    
        # Noisy estimate of position
        belief_state = boresight_polar_2_cartesian(range, steer_ang + randn() * beamwidth / 2)
    
        return belief_state
    end,


    reward = function (s, a, belief_state)
        xpos, ypos = s
        steer_ang, beamwidth, tx_power = a
    
        # Tracking error (Euclidean)
        tracking_error = sqrt((xpos - belief_state[1])^2 + (ypos - belief_state[2])^2)
    
        if tracking_error < 100
            return 100.0
        else
            # Normalized costs
            power_cost = tx_power / 100e3
            norm_tracking_error = tracking_error / sqrt(2 * (x_max_size^2 + y_max_size^2))
            return -100 * (power_cost + norm_tracking_error)
        end
    end,


    # ========================
        # INITIAL STATE GENERATION
    # ========================
    # statetype = Vector{Float64},  # State: [x, y]
    
    # initialstate = ImplicitDistribution([10000*randn(), 10000*randn()]),

    initialstate = function()

        init_x = 10000*randn()
        init_y = 10000*randn()

        # Reflector
        pos_ref = SVector(init_x, init_y)
        snapped_pos_ref = snap_to_grid(env.grid_x, pos_ref)
        rx_ref = Reflector(snapped_pos_ref, 0.9, 10^10, false)
        add_reflector!(env, rx_ref)

        return ImplicitDistribution(rng -> [init_x, init_y])
    end,

    isterminal = s -> false  # Tracking problem — no natural terminal state 
    # TODO: termainal states will be when the target reaches the end of the radar simulator
)


QuickPOMDP{Base.UUID("0d99a56f-8f3e-47c0-a533-e7fda9d39b91"), Vector{Float64}, Tuple{Int64, Int64, Int64}, Vector{Float64}, @NamedTuple{isterminal::var"#439#448", actionindex::Dict{Tuple{Int64, Int64, Int64}, Int64}, statetype::DataType, transition::var"#434#443", actions::Vector{Tuple{Int64, Int64, Int64}}, reward::var"#436#445", discount::Float64, initialstate::var"#437#446", obstype::DataType, observation::var"#435#444"}}((isterminal = var"#439#448"(), actionindex = Dict((75, 20, 100) => 556, (160, 40, 100) => 1184, (35, 45, 100) => 288, (40, 15, 10) => 298, (105, 20, 50) => 771, (240, 45, 50) => 1763, (165, 5, 1) => 1189, (215, 15, 50) => 1559, (85, 45, 10) => 646, (125, 30, 50) => 923…), statetype = Vector{Float64}, transition = var"#434#443"(), actions = [(0, 5, 1), (0, 5, 10), (0, 5, 50), (0, 5, 100), (0, 10, 1), (0, 10, 10), (0, 10, 50), (0, 10, 100), (0, 15, 1), (0, 15, 10)  …  (355, 35, 50), (355, 35, 100), (355, 40, 1), (355, 40, 10), (355, 40, 50), (355, 40, 100), (355, 45,

In [122]:
@show discount(radar)
@show actions(radar)[2000]
@show rand(initialstate(radar)) 

discount(radar) = 0.95
(actions(radar))[2000] = (275, 25, 100)
rand(initialstate(radar)) = [3091.540750157148, 6702.86615225249]


2-element Vector{Float64}:
 3091.540750157148
 6702.86615225249

In [ ]:
# Test evaluation: policy alway looks forward
policy = FunctionPolicy(o->POMDPs.actions(radar)[1])
sim = RolloutSimulator(max_steps=10)
POMDPs.simulate(sim, radar, policy)

## 2. Create Updater (Particle Filter)